# Querying

This notebook demonstrates Nexus Forge data [querying features](https://nexus-forge.readthedocs.io/en/latest/interaction.html#querying).

In [1]:
from kgforge.core import KnowledgeGraphForge

A configuration file is needed in order to create a KnowledgeGraphForge session. A configuration can be generated using the notebook [00-Initialization.ipynb](00%20-%20Initialization.ipynb).

In [2]:
forge = KnowledgeGraphForge("../../configurations/forge.yml")

## Imports

In [3]:
from kgforge.core import Resource
from kgforge.specializations.resources import Dataset
from kgforge.core.wrappings.paths import Filter, FilterOperator

## Retrieval

### latest version

In [4]:
jane = Resource(type="Person", name="Jane Doe", award=["Nobel"])

In [5]:
forge.register(jane)

<action> _register_one
<succeeded> True


In [6]:
resource = forge.retrieve(jane.id)

In [7]:
resource == jane

True

### specific version

In [8]:
jane = Resource(type="Person", name="Jane Doe", award=["Nobel"])

In [9]:
forge.register(jane)

<action> _register_one
<succeeded> True


In [10]:
forge.tag(jane, "v1")

<action> _tag_one
<succeeded> True


In [11]:
jane.email = ["jane.doe@epfl.ch", "jane.doe@example.org"]

In [12]:
forge.update(jane)

<action> _update_one
<succeeded> True


In [13]:
try:
    # DemoStore
    print(jane._store_metadata.version)
except:
    # BlueBrainNexus
    print(jane._store_metadata._rev)

3


In [14]:
jane_v1 = forge.retrieve(jane.id, version=1)

In [15]:
jane_v1_tag = forge.retrieve(jane.id, version="v1")

In [16]:
jane_v1_rev = forge.retrieve(jane.id+"?rev=1")

In [17]:
jane_v1 == jane_v1_tag

True

In [18]:
jane_v1 == jane_v1_rev

True

In [19]:
jane_v1 != jane

True

In [20]:
try:
    # DemoStore
    print(jane_v1._store_metadata.version)
except:
    # BlueBrainNexus
    print(jane_v1._store_metadata._rev)

1


### crossbucket retrieval
It is possible to retrieve resources stored in buckets different then the configured one. The configured store should of course support it.

In [21]:
resource = forge.retrieve(jane.id, cross_bucket=True) # cross_bucket defaults to False

In [22]:
resource._store_metadata

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/4088d652-7b1c-42a9-9570-97ca23f4ec98',
 '_constrainedBy': 'https://bluebrain.github.io/nexus/schemas/unconstrained.json',
 '_createdAt': '2023-05-22T08:57:55.754Z',
 '_createdBy': 'https://sandbox.bluebrainnexus.io/v1/realms/github/users/crisely09',
 '_deprecated': False,
 '_incoming': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/4088d652-7b1c-42a9-9570-97ca23f4ec98/incoming',
 '_outgoing': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/4088d652-7b1c-42a9-9570-97ca23f4ec98/outgoing',
 '_project': 'https://sandbox.bluebrainnexus.io/v1/projects/github-users/crisely09',
 '_rev': 3,
 '_schemaProject': 'https://sandbox.bluebrainnexus.io/v1/projects/github-users/crisely09',
 '_self': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/4088d652-7b1c-42a9-9570-97ca23f4ec98',
 '_updatedAt': '2023-05-22T08:57:56.380Z',
 '_updatedBy': 'https://

In [23]:
resource._last_action

Action(error=None, message=None, operation='retrieve', succeeded=True)

In [24]:
resource._synchronized

True

### Retrieving using the resorce url
One can also use the value of `_self` from ._stote_metadata to retrieve a resource

In [25]:
import copy

In [26]:
other_resource = copy.deepcopy(resource)

In [27]:
other_resource.id = "https://myincreadibleid-987654321"

In [28]:
forge.register(other_resource)

<action> _register_one
<succeeded> True


In [29]:
url = other_resource._store_metadata['_self']

In [30]:
same_resource_url = forge.retrieve(id=url)
same_resource_id = forge.retrieve(id=other_resource.id)

Confirm they are the same

In [132]:
same_resource_id == same_resource_url

True

### Original source retrieval
When using BlueBrainNexusStore, it is possible to retrieve resources' payload as they were registered (retrieve_source=True) without any changes related to store added metadata or JSONLD framing.

In [34]:
resource = forge.retrieve(jane.id, retrieve_source=False) # retrieve_source defaults to True

In [35]:
forge.as_json(resource)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/4088d652-7b1c-42a9-9570-97ca23f4ec98',
 'type': 'Person',
 'award': 'Nobel',
 'email': ['jane.doe@epfl.ch', 'jane.doe@example.org'],
 'name': 'Jane Doe'}

In [36]:
resource._store_metadata

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/4088d652-7b1c-42a9-9570-97ca23f4ec98',
 '_constrainedBy': 'https://bluebrain.github.io/nexus/schemas/unconstrained.json',
 '_createdAt': '2023-05-22T08:57:55.754Z',
 '_createdBy': 'https://sandbox.bluebrainnexus.io/v1/realms/github/users/crisely09',
 '_deprecated': False,
 '_incoming': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/4088d652-7b1c-42a9-9570-97ca23f4ec98/incoming',
 '_outgoing': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/4088d652-7b1c-42a9-9570-97ca23f4ec98/outgoing',
 '_project': 'https://sandbox.bluebrainnexus.io/v1/projects/github-users/crisely09',
 '_rev': 3,
 '_schemaProject': 'https://sandbox.bluebrainnexus.io/v1/projects/github-users/crisely09',
 '_self': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/4088d652-7b1c-42a9-9570-97ca23f4ec98',
 '_updatedAt': '2023-05-22T08:57:56.380Z',
 '_updatedBy': 'https://

In [37]:
resource._last_action

Action(error=None, message=None, operation='retrieve', succeeded=True)

In [38]:
resource._synchronized

True

### error handling

In [39]:
resource = forge.retrieve("123")

<action> raise_for_status
<error> HTTPError: 404 Client Error: Not Found for url: https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/%3A%2F%2F123



In [40]:
resource is None

True

## Searching

Note: DemoModel and RdfModel schemas have not been synchronized yet. This section is to be run with RdfModel. Commented lines are for DemoModel.

In [41]:
jane = Resource(type="Person", name="Jane Doe")
contribution_jane = Resource(type="Contribution", agent=jane)

In [42]:
john = Resource(type="Person", name="John Smith")
contribution_john = Resource(type="Contribution", agent=john)

In [43]:
dataset = Dataset(forge, type="Dataset", contribution=[contribution_jane, contribution_john])
dataset.add_distribution("../../data/associations.tsv")

In [44]:
forge.register(dataset)

<action> _register_one
<succeeded> True


In [45]:
forge.as_json(dataset)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/5b59d793-d0de-42ec-a841-eac6e06707db',
 'type': 'Dataset',
 'contribution': [{'type': 'Contribution',
   'agent': {'type': 'Person', 'name': 'Jane Doe'}},
  {'type': 'Contribution', 'agent': {'type': 'Person', 'name': 'John Smith'}}],
 'distribution': {'type': 'DataDownload',
  'atLocation': {'type': 'Location',
   'store': {'id': 'https://bluebrain.github.io/nexus/vocabulary/diskStorageDefault',
    'type': 'DiskStorage',
    '_rev': 1}},
  'contentSize': {'unitCode': 'bytes', 'value': 477},
  'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/crisely09/1f6d9e0d-a4e3-46de-b742-3665af2c6b53',
  'digest': {'algorithm': 'SHA-256',
   'value': '789aa07948683fe036ac29811814a826b703b562f7d168eb70dee1fabde26859'},
  'encodingFormat': 'text/tab-separated-values',
  'name': 'associations.tsv'}}

### Using resource paths as filters

The `paths` method load the template or property paths (ie. expected properties) for a given type.

Please refer to the [Modeling.ipynb](11%20-%20Modeling.ipynb) notebook to learn about templates and types.

In [46]:
p = forge.paths("Dataset")

Autocompletion is enabled on `p` and this can be used to create search filters.

Note: There is a known issue for RdfModel which requires using `p.type.id` instead of `p.type`.

All [python comparison operators](https://www.w3schools.com/python/gloss_python_comparison_operators.asp) are supported.

In [47]:
resources = forge.search(p.type.id=="Person", limit=3)

In [48]:
type(resources)

list

In [49]:
len(resources)

3

In [50]:
forge.as_dataframe(resources)

,id,type,name,distribution.type,distribution.atLocation.type,distribution.atLocation.store.id,distribution.atLocation.store.type,distribution.atLocation.store._rev,distribution.contentSize.unitCode,distribution.contentSize.value,distribution.contentUrl,distribution.digest.algorithm,distribution.digest.value,distribution.encodingFormat,distribution.name
0,https://sandbox.bluebrainnexus.io/v1/resources...,Person,Albert Einstein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://sandbox.bluebrainnexus.io/v1/resources...,Person,Marie Curie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://sandbox.bluebrainnexus.io/v1/resources...,Person,Jane Doe,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1.0,bytes,477.0,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,789aa07948683fe036ac29811814a826b703b562f7d168...,text/tab-separated-values,associations.tsv


In [51]:
forge.as_dataframe(resources, store_metadata=True)

,id,type,name,_constrainedBy,_createdAt,_createdBy,_deprecated,_incoming,_outgoing,_project,...,distribution.atLocation.store.id,distribution.atLocation.store.type,distribution.atLocation.store._rev,distribution.contentSize.unitCode,distribution.contentSize.value,distribution.contentUrl,distribution.digest.algorithm,distribution.digest.value,distribution.encodingFormat,distribution.name
0,https://sandbox.bluebrainnexus.io/v1/resources...,Person,Albert Einstein,https://bluebrain.github.io/nexus/schemas/unco...,2022-07-20T15:30:01.482Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://sandbox.bluebrainnexus.io/v1/resources...,Person,Marie Curie,https://bluebrain.github.io/nexus/schemas/unco...,2022-07-20T15:30:01.467Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://sandbox.bluebrainnexus.io/v1/resources...,Person,Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2022-07-20T15:27:50.435Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,...,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1.0,bytes,477.0,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,789aa07948683fe036ac29811814a826b703b562f7d168...,text/tab-separated-values,associations.tsv


In [52]:
# Search results are not synchronized
resources[0]._synchronized

False

#### Using nested resource property

Property autocompletion is available on a path `p` even for nested properties like `p.contribution`.

In [53]:
# Search for resources of type Dataset and with text/tab-separated-values as distribution.encodingFormat
resources = forge.search(p.type.id == "Dataset", p.distribution.encodingFormat == "text/tab-separated-values", limit=3)

In [54]:
len(resources)

3

In [55]:
forge.as_dataframe(resources)

,id,type,contribution,distribution.type,distribution.atLocation.type,distribution.atLocation.store.id,distribution.atLocation.store.type,distribution.atLocation.store._rev,distribution.contentSize.unitCode,distribution.contentSize.value,distribution.contentUrl,distribution.digest.algorithm,distribution.digest.value,distribution.encodingFormat,distribution.name
0,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,"[{'type': 'Contribution', 'agent': {'type': 'P...",DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,789aa07948683fe036ac29811814a826b703b562f7d168...,text/tab-separated-values,associations.tsv
1,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,"[{'type': 'Contribution', 'agent': {'type': 'P...",DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,789aa07948683fe036ac29811814a826b703b562f7d168...,text/tab-separated-values,associations.tsv
2,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,"[{'type': 'Contribution', 'agent': {'type': 'P...",DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,789aa07948683fe036ac29811814a826b703b562f7d168...,text/tab-separated-values,associations.tsv


### Using dictionaries as filters
A dictionary can be provided for filters:
* {'type': {'id':'Dataset'}} is equivalent to p.type.id=="Dataset"
* only the '==' operator is supported
* nested dict are supported
* it is not mandatory for the provided properties and values to be defined in the forge model. Results will be retrieved if there are corresponding data in the store.

This feature is not supported when using the DemoStore


In [56]:
# Search for resources of type Dataset and with text/tab-separated-values as distribution.encodingFormat
# and created a given dateTime (by default, dateTime values should be signaled by the suffix "^^xsd:dateTime")
filters = {
           "type": "Dataset", 
           "distribution":{"encodingFormat":"text/tab-separated-values"},
           "_createdAt":dataset._store_metadata._createdAt+"^^xsd:dateTime"
          }
resources = forge.search(filters, limit=3)

In [57]:
type(resources)

list

In [58]:
len(resources)

1

In [59]:
forge.as_dataframe(resources, store_metadata=True)

,id,type,contribution,distribution.type,distribution.atLocation.type,distribution.atLocation.store.id,distribution.atLocation.store.type,distribution.atLocation.store._rev,distribution.contentSize.unitCode,distribution.contentSize.value,...,_createdBy,_deprecated,_incoming,_outgoing,_project,_rev,_schemaProject,_self,_updatedAt,_updatedBy
0,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,"[{'type': 'Contribution', 'agent': {'type': 'P...",DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,...,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,1,https://sandbox.bluebrainnexus.io/v1/projects/...,https://sandbox.bluebrainnexus.io/v1/resources...,2023-05-22T08:58:03.175Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...


### Using built-in Filter objects

#### Supported filter operators

In [60]:
[f"{op.value} ({op.name})" for op in FilterOperator] # These are equivalent to the Python comparison operators

['__eq__ (EQUAL)',
 '__ne__ (NOT_EQUAL)',
 '__lt__ (LOWER_THAN)',
 '__le__ (LOWER_OR_Equal_Than)',
 '__gt__ (GREATER_Than)',
 '__ge__ (GREATER_OR_Equal_Than)']

In [61]:
# Search for resources of type Dataset and with text/tab-separated-values as distribution.encodingFormat
# and created a given dateTime (dateTime values should be signaled by the suffix "^^xsd:dateTime")
filter_1 = Filter(operator=FilterOperator.EQUAL, path=["type"], value="Dataset")
filter_2 = Filter(operator=FilterOperator.EQUAL, path=["distribution","encodingFormat"], value="text/tab-separated-values")
filter_3 = Filter(operator=FilterOperator.LOWER_OR_Equal_Than, path=["_createdAt"], value=dataset._store_metadata._createdAt+"^^xsd:dateTime")

resources = forge.search(filter_1, filter_2, filter_3, limit=3)

In [62]:
type(resources)

list

In [63]:
len(resources)

3

In [64]:
forge.as_dataframe(resources, store_metadata=True)

,id,type,contribution,distribution.type,distribution.atLocation.type,distribution.atLocation.store.id,distribution.atLocation.store.type,distribution.atLocation.store._rev,distribution.contentSize.unitCode,distribution.contentSize.value,...,_createdBy,_deprecated,_incoming,_outgoing,_project,_rev,_schemaProject,_self,_updatedAt,_updatedBy
0,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,"[{'type': 'Contribution', 'agent': {'type': 'P...",DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,...,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,1,https://sandbox.bluebrainnexus.io/v1/projects/...,https://sandbox.bluebrainnexus.io/v1/resources...,2023-03-29T12:57:29.375Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...
1,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,"[{'type': 'Contribution', 'agent': {'type': 'P...",DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,...,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,1,https://sandbox.bluebrainnexus.io/v1/projects/...,https://sandbox.bluebrainnexus.io/v1/resources...,2023-05-22T08:24:43.121Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...
2,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,"[{'type': 'Contribution', 'agent': {'type': 'P...",DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1,bytes,477,...,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,1,https://sandbox.bluebrainnexus.io/v1/projects/...,https://sandbox.bluebrainnexus.io/v1/resources...,2023-05-22T08:58:03.175Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...


### Using search endpoints

Two types of search endpoints are supported: 'sparql' (default) for graph queries and 'elastic' for document oriented queries. The types of available search endpoint can be configured (see [00-Initialization.ipynb](00%20-%20Initialization.ipynb) for an example of search endpoints config) or set when creating a KnowledgeGraphForge session using the 'searchendpoints' arguments.

The search endpoint to hit when calling forge.search(...) is 'sparql' by default but can be specified using the 'search_endpoint' argument.

#### SPARQL Search Endpoint

In [65]:
# Search for resources of type Person
filters = {"type": "Person"}
resources = forge.search(filters, limit=3, search_endpoint='sparql')

In [66]:
type(resources)

list

In [67]:
len(resources)

3

In [68]:
forge.as_dataframe(resources, store_metadata=True)

,id,type,name,_constrainedBy,_createdAt,_createdBy,_deprecated,_incoming,_outgoing,_project,...,distribution.atLocation.store.id,distribution.atLocation.store.type,distribution.atLocation.store._rev,distribution.contentSize.unitCode,distribution.contentSize.value,distribution.contentUrl,distribution.digest.algorithm,distribution.digest.value,distribution.encodingFormat,distribution.name
0,https://sandbox.bluebrainnexus.io/v1/resources...,Person,Albert Einstein,https://bluebrain.github.io/nexus/schemas/unco...,2022-07-20T15:30:01.482Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://sandbox.bluebrainnexus.io/v1/resources...,Person,Marie Curie,https://bluebrain.github.io/nexus/schemas/unco...,2022-07-20T15:30:01.467Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://sandbox.bluebrainnexus.io/v1/resources...,Person,Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2022-07-20T15:27:50.435Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,...,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1.0,bytes,477.0,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,789aa07948683fe036ac29811814a826b703b562f7d168...,text/tab-separated-values,associations.tsv


#### ElasticSearch Endpoint

In [69]:
# Search for resources of type Person and retrieve their ids and names.
filters = {"@type": "http://schema.org/Person"}
resources = forge.search(filters, limit=3, 
                         search_endpoint='elastic', 
                         includes=["@id", "@type"]) # fields can also be excluded with 'excludes'

In [70]:
type(resources)

list

In [71]:
len(resources)

3

In [72]:
forge.as_dataframe(resources, store_metadata=True)

,id,type
0,https://sandbox.bluebrainnexus.io/v1/resources...,http://schema.org/Person
1,https://sandbox.bluebrainnexus.io/v1/resources...,http://schema.org/Person
2,https://sandbox.bluebrainnexus.io/v1/resources...,http://schema.org/Person


In [73]:
# Search results are not synchronized
resources[0]._synchronized

False

In [74]:
resources[0].id

'https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/e491b538-0fc4-4fae-b5f7-aaf80839dbe5'

In [75]:
resources[0].type

'http://schema.org/Person'

### Crossbucket search
It is possible to search for resources stored in buckets different than the configured one. The configured store should of course support it.

In [76]:
resources = forge.search(p.type.id == "Association", limit=3, cross_bucket=True)  # cross_bucket defaults to False

In [77]:
type(resources)

list

In [78]:
len(resources)

3

In [79]:
forge.as_dataframe(resources)

,id,type,agent.type,agent.gender.id,agent.gender.type,agent.gender.label,agent.name,distribution.type,distribution.atLocation.type,distribution.atLocation.store.id,distribution.atLocation.store.type,distribution.atLocation.store._rev,distribution.contentSize.unitCode,distribution.contentSize.value,distribution.contentUrl,distribution.digest.algorithm,distribution.digest.value,distribution.encodingFormat,distribution.name,name
0,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,http://purl.obolibrary.org/obo/PATO_0000383,LabeledOntologyEntity,female,Marie Curie,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1.0,bytes,46.0,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,e0fe65f725bf28fe2b88c7bafb51fb5ef1df0ab14c68a3...,text/plain,marie_curie.txt,Curie Association
1,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,http://purl.obolibrary.org/obo/PATO_0000384,LabeledOntologyEntity,male,Albert Einstein,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1.0,bytes,50.0,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,91a5ce5c84dc5bead730a4b49d0698b4aaef4bc06ce164...,text/plain,albert_einstein.txt,Einstein Association
2,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,NaN,NaN,NaN,John Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
#Furthermore it is possible to filter by bucket when cross_bucket is set to True. Setting a bucket value when cross_bucket is False will trigger a not_supported exception.
resources = forge.search(p.type.id == "Person", limit=3, cross_bucket=True, bucket="dke/kgforge")  # add a bucket

In [81]:
type(resources)

list

In [82]:
len(resources)

0

In [83]:
forge.as_dataframe(resources)

""


### Searching original source
When using BlueBrainNexusStore, it is possible to retrieve resources' payload as they were registered (retrieve_source=True) without any changes related to store added metadata or JSONLD framing.

In [84]:
resources = forge.search(p.type.id == "Association", limit=3, retrieve_source=False)  # retrieve_source defaults to True

In [85]:
type(resources)

list

In [86]:
len(resources)

3

In [87]:
forge.as_dataframe(resources)

,id,type,agent.type,agent.gender.id,agent.gender.type,agent.gender.label,agent.name,distribution.type,distribution.atLocation.type,distribution.atLocation.store.id,distribution.atLocation.store.type,distribution.atLocation.store._rev,distribution.contentSize.unitCode,distribution.contentSize.value,distribution.contentUrl,distribution.digest.algorithm,distribution.digest.value,distribution.encodingFormat,distribution.name,name
0,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,http://purl.obolibrary.org/obo/PATO_0000383,LabeledOntologyEntity,female,Marie Curie,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1.0,bytes,46.0,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,e0fe65f725bf28fe2b88c7bafb51fb5ef1df0ab14c68a3...,text/plain,marie_curie.txt,Curie Association
1,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,http://purl.obolibrary.org/obo/PATO_0000384,LabeledOntologyEntity,male,Albert Einstein,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,DiskStorage,1.0,bytes,50.0,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,91a5ce5c84dc5bead730a4b49d0698b4aaef4bc06ce164...,text/plain,albert_einstein.txt,Einstein Association
2,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,NaN,NaN,NaN,John Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Graph traversing

SPARQL is used as a query language to perform graph traversing.

Nexus Forge implements a SPARQL query rewriting strategy leveraging a configured RDFModel that lets users write SPARQL queries without adding prefix declarations, prefix names or long IRIs. With this strategy, only type and property names can be provided.

Please refer to the [Modeling.ipynb](11%20-%20Modeling.ipynb) notebook to learn about templates.

Note: DemoStore doesn't implement SPARQL operations yet. Please use another store for this section.

Note: DemoModel and RdfModel schemas have not been synchronized yet. This section is to be run with RdfModel.

In [88]:
jane = Resource(type="Person", name="Jane Doe")
contribution_jane = Resource(type="Contribution", agent=jane)

In [89]:
john = Resource(type="Person", name="John Smith")
contribution_john = Resource(type="Contribution", agent=john)

In [90]:
association = Resource(type="Dataset", contribution=[contribution_jane, contribution_john])

In [91]:
forge.register(association)

<action> _register_one
<succeeded> True


In [92]:
forge.template("Dataset") # Templates help know which property to use when writing a query to serach for a given type

{
    id: ""
    type:
    {
        id: ""
    }
    annotation:
    {
        id: ""
        type: Annotation
        hasBody:
        {
            id: ""
            type:
            {
                id: ""
            }
            label: ""
            note: ""
        }
        hasTarget:
        {
            id: ""
            type: AnnotationTarget
        }
        note: ""
    }
    brainLocation:
    {
        id: ""
        type: BrainLocation
        atlasSpatialReferenceSystem:
        {
            id: ""
            type: AtlasSpatialReferenceSystem
        }
        brainRegion:
        {
            id: ""
            label: ""
        }
        coordinatesInBrainAtlas:
        {
            id: ""
            valueX: 0.0
            valueY: 0.0
            valueZ: 0.0
        }
        coordinatesInSlice:
        {
            spatialReferenceSystem:
            {
                id: ""
                type: SpatialReferenceSystem
            }
            valueX

### Prefix and namespace free SPARQL query

When a forge RDFModel is configured, then there is no need to provide prefixes and namespaces when writing a SPARQL query. Prefixes and namespaces will be automatically inferred from the provided schemas and/or JSON-LD context and the query rewritten accordingly.

In [93]:
query = """
    SELECT ?id ?name ?contributor
    WHERE {
        ?id a Dataset ;
        contribution/agent ?contributor.
        ?contributor name ?name.
    }
"""

In [94]:
resources = forge.sparql(query, limit=3)

In [95]:
type(resources)

list

In [96]:
len(resources)

3

In [97]:
print(resources[0])

{
    id: https://sandbox.bluebrainnexus.io/v1/resources/github-users/crisely09/_/40162355-af23-44a9-9c9a-36b6a4d35dcf
    contributor: t3318
    name: Jane Doe
}


In [98]:
forge.as_dataframe(resources)

,id,contributor,name
0,https://sandbox.bluebrainnexus.io/v1/resources...,t3318,Jane Doe
1,https://sandbox.bluebrainnexus.io/v1/resources...,t3319,John Smith
2,https://sandbox.bluebrainnexus.io/v1/resources...,t3325,Jane Doe


### display rewritten SPARQL query 

In [99]:
resources = forge.sparql(query, limit=3, debug=True)

Submitted query:
   PREFIX dc: <http://purl.org/dc/elements/1.1/>
   PREFIX dcat: <http://www.w3.org/ns/dcat#>
   PREFIX dcterms: <http://purl.org/dc/terms/>
   PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
   PREFIX nsg: <https://neuroshapes.org/>
   PREFIX owl: <http://www.w3.org/2002/07/owl#>
   PREFIX prov: <http://www.w3.org/ns/prov#>
   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX schema: <http://schema.org/>
   PREFIX sh: <http://www.w3.org/ns/shacl#>
   PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
   PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
   PREFIX vann: <http://purl.org/vocab/vann/>
   PREFIX void: <http://rdfs.org/ns/void#>
   PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
   PREFIX : <https://neuroshapes.org/>
   
       SELECT ?id ?name ?contributor
       WHERE {
           ?id a schema:Dataset ;
           nsg:contribution/prov:agent ?contributor.
           ?c

### Full SPARQL query

Regular SPARQL query can also be provided. When provided, the limit and offset arguments superseed any in query limit or offset values.

In [100]:
query = """
PREFIX dc: <http://purl.org/dc/elements/1.1/>
   PREFIX dcat: <http://www.w3.org/ns/dcat#>
   PREFIX dcterms: <http://purl.org/dc/terms/>
   PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
   PREFIX nsg: <https://neuroshapes.org/>
   PREFIX owl: <http://www.w3.org/2002/07/owl#>
   PREFIX prov: <http://www.w3.org/ns/prov#>
   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX schema: <http://schema.org/>
   PREFIX sh: <http://www.w3.org/ns/shacl#>
   PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
   PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
   PREFIX vann: <http://purl.org/vocab/vann/>
   PREFIX void: <http://rdfs.org/ns/void#>
   PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
   PREFIX : <https://neuroshapes.org/>
   SELECT ?id ?name
   WHERE {
       ?id a schema:Dataset ;
       nsg:contribution/prov:agent ?contributor.
       ?contributor schema:name ?name.
   }
   ORDER BY ?id
   LIMIT 1
   OFFSET 0
"""

In [101]:
# it is recommended to set 'rewrite' to 'False' to prevent the sparql query rewriting when a syntactically correct SPARQL query is provided.
resources = forge.sparql(query, rewrite=False, limit=3, offset=1, debug=True) 

Submitted query:
   
   PREFIX dc: <http://purl.org/dc/elements/1.1/>
      PREFIX dcat: <http://www.w3.org/ns/dcat#>
      PREFIX dcterms: <http://purl.org/dc/terms/>
      PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
      PREFIX nsg: <https://neuroshapes.org/>
      PREFIX owl: <http://www.w3.org/2002/07/owl#>
      PREFIX prov: <http://www.w3.org/ns/prov#>
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      PREFIX schema: <http://schema.org/>
      PREFIX sh: <http://www.w3.org/ns/shacl#>
      PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
      PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
      PREFIX vann: <http://purl.org/vocab/vann/>
      PREFIX void: <http://rdfs.org/ns/void#>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
      PREFIX : <https://neuroshapes.org/>
      SELECT ?id ?name
      WHERE {
          ?id a schema:Dataset ;
          nsg:contribution/prov:ag

In [102]:
type(resources)

list

In [103]:
len(resources)

3

In [104]:
type(resources[0])

kgforge.core.resource.Resource

In [105]:
forge.as_dataframe(resources)

,id,name
0,https://sandbox.bluebrainnexus.io/v1/resources...,John Smith
1,https://sandbox.bluebrainnexus.io/v1/resources...,John Smith
2,https://sandbox.bluebrainnexus.io/v1/resources...,Jane Doe


### Avoid rewriting the query

To not assign any limit or offset, one can pass `None` to those parameters

In [106]:
query_without_limit = """
   PREFIX dc: <http://purl.org/dc/elements/1.1/>
   PREFIX dcat: <http://www.w3.org/ns/dcat#>
   PREFIX dcterms: <http://purl.org/dc/terms/>
   PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
   PREFIX nsg: <https://neuroshapes.org/>
   PREFIX owl: <http://www.w3.org/2002/07/owl#>
   PREFIX prov: <http://www.w3.org/ns/prov#>
   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX schema: <http://schema.org/>
   PREFIX sh: <http://www.w3.org/ns/shacl#>
   PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
   PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
   PREFIX vann: <http://purl.org/vocab/vann/>
   PREFIX void: <http://rdfs.org/ns/void#>
   PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
   PREFIX : <https://neuroshapes.org/>
   SELECT ?id ?name
   WHERE {
       ?id a schema:Dataset ;
       nsg:contribution/prov:agent ?contributor.
       ?contributor schema:name ?name.
   }
   ORDER BY ?id
"""

In [107]:

resources = forge.sparql(query_without_limit, rewrite=False, limit=None, offset=None, debug=True)

Submitted query:
   
      PREFIX dc: <http://purl.org/dc/elements/1.1/>
      PREFIX dcat: <http://www.w3.org/ns/dcat#>
      PREFIX dcterms: <http://purl.org/dc/terms/>
      PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
      PREFIX nsg: <https://neuroshapes.org/>
      PREFIX owl: <http://www.w3.org/2002/07/owl#>
      PREFIX prov: <http://www.w3.org/ns/prov#>
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      PREFIX schema: <http://schema.org/>
      PREFIX sh: <http://www.w3.org/ns/shacl#>
      PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
      PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
      PREFIX vann: <http://purl.org/vocab/vann/>
      PREFIX void: <http://rdfs.org/ns/void#>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
      PREFIX : <https://neuroshapes.org/>
      SELECT ?id ?name
      WHERE {
          ?id a schema:Dataset ;
          nsg:contribution/prov

In [108]:
len(resources)

100

If you only want to add the context, but keep the content of the query the same, you need to set the `rewrite` parameter to `False`

In [109]:
query_without_context = """
    SELECT ?id ?name ?contributor
    WHERE {
        ?id a Dataset ;
        contribution/agent ?contributor.
        ?contributor name ?name.
    }
"""

In [110]:
resources = forge.sparql(query_without_context, limit=None, debug=True)

Submitted query:
   PREFIX dc: <http://purl.org/dc/elements/1.1/>
   PREFIX dcat: <http://www.w3.org/ns/dcat#>
   PREFIX dcterms: <http://purl.org/dc/terms/>
   PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
   PREFIX nsg: <https://neuroshapes.org/>
   PREFIX owl: <http://www.w3.org/2002/07/owl#>
   PREFIX prov: <http://www.w3.org/ns/prov#>
   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX schema: <http://schema.org/>
   PREFIX sh: <http://www.w3.org/ns/shacl#>
   PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
   PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
   PREFIX vann: <http://purl.org/vocab/vann/>
   PREFIX void: <http://rdfs.org/ns/void#>
   PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
   PREFIX : <https://neuroshapes.org/>
   
       SELECT ?id ?name ?contributor
       WHERE {
           ?id a schema:Dataset ;
           nsg:contribution/prov:agent ?contributor.
           ?c

In [111]:
len(resources)

100

## ElasticSearch DSL Query

ElasticSearch DSL can be used as a query language search for resources provided that the configured store supports it. The 'BlueBrainNexusStore' supports ElasticSearch.

Note: DemoStore doesn't implement ElasaticSearch DSL operations.

In [112]:
jane = Resource(type="Person", name="Jane Doe")
contribution_jane = Resource(type="Contribution", agent=jane)

In [113]:
john = Resource(type="Person", name="John Smith")
contribution_john = Resource(type="Contribution", agent=john)

In [114]:
association = Resource(type="Dataset", contribution=[contribution_jane, contribution_john])

In [115]:
forge.register(association)

<action> _register_one
<succeeded> True


### Plain ElasticSearch DSL

In [116]:
query = """
        {
          "_source": {
            "includes": [
              "@id",
              "name"
            ]
          },
          "query": {
            "term": {
              "@type": "http://schema.org/Dataset"
            }
          }
        }
"""

In [117]:
# limit and offset (when provided in this method call) superseed 'size' and 'from' values provided in the query
resources = forge.elastic(query, limit=3)

In [118]:
type(resources)

list

In [119]:
len(resources)

3

In [120]:
type(resources[0])

kgforge.core.resource.Resource

In [121]:
forge.as_dataframe(resources)

,id,name
0,https://sandbox.bluebrainnexus.io/v1/resources...,Interesting associations
1,https://sandbox.bluebrainnexus.io/v1/resources...,Interesting people
2,https://sandbox.bluebrainnexus.io/v1/resources...,Interesting people


## Downloading

Note: DemoStore doesn't implement file operations yet. Please use another store for this section.

In [122]:
jane = Resource(type="Person", name="Jane Doe")

In [123]:
! ls -p ../../data | egrep -v /$

associations.tsv
my_data.xwz
my_data_derived.txt
persons-with-id.csv
persons.csv
tfidfvectorizer_model_schemaorg_linking


In [124]:
distribution = forge.attach("../../data")

In [125]:
association = Resource(type="Association", agent=jane, distribution=distribution)

In [126]:
forge.register(association)

<action> _register_one
<succeeded> True


In [127]:
# By default, the downladable file urls are collected from the json path "distribution.contentUrl" (follow="distribution.contentUrl") and
# the files are downloaded in the current path (path="."). 
# The argument overwrite: bool can be provided to decide whether to overwrite (True) existing files with the same name or
# to create new ones (False) with their names suffixed with a timestamp.
# A cross_bucket argument can be provided to download data from the configured bucket (cross_bucket=False - the default value) 
# or from a bucket different than the configured one (cross_bucket=True). The configured store should support crossing buckets for this to work.
forge.download(association)

In [128]:
# Specific content type can be downloaded.
forge.download(association, content_type="text/tab-separated-values")

In [129]:
# The urls or the files to download can be collected from a different json path (by setting a value for "follow") and 
# the files downloaded to a different path (by setting a value for "path")
forge.download(association, follow="distribution.contentUrl", path="./downloaded/")

In [130]:
! ls -l ./downloaded/

total 896
-rw-r--r--  1 cgonzale  10067     477 May 22 10:24 associations.tsv
-rw-r--r--  1 cgonzale  10067     477 May 22 10:58 associations.tsv.20230522105817
-rw-r--r--  1 cgonzale  10067      16 May 22 10:24 my_data.xwz
-rw-r--r--  1 cgonzale  10067      16 May 22 10:58 my_data.xwz.20230522105817
-rw-r--r--  1 cgonzale  10067      24 May 22 10:24 my_data_derived.txt
-rw-r--r--  1 cgonzale  10067      24 May 22 10:58 my_data_derived.txt.20230522105817
-rw-r--r--  1 cgonzale  10067     126 May 22 10:24 persons-with-id.csv
-rw-r--r--  1 cgonzale  10067     126 May 22 10:58 persons-with-id.csv.20230522105817
-rw-r--r--  1 cgonzale  10067      52 May 22 10:24 persons.csv
-rw-r--r--  1 cgonzale  10067      52 May 22 10:58 persons.csv.20230522105817
-rw-r--r--  1 cgonzale  10067  204848 May 22 10:24 tfidfvectorizer_model_schemaorg_linking
-rw-r--r--  1 cgonzale  10067  204848 May 22 10:58 tfidfvectorizer_model_schemaorg_linking.20230522105817


In [131]:
#! rm -R ./downloaded/